In [1]:
import numpy
import pandas
import matplotlib.pyplot as plt

In [2]:
def cosine(x):
    return (numpy.cos(3 * x) + 1) / 2

def gaussian(x):
    return numpy.exp(-0.25 * x ** 2)

def polynomial(x):
    return (x ** 5 + 3 * x ** 4 - 11 * x ** 3 - 27 * x ** 2 + 10 * x + 64) / 100 

def sigmoid(x):
    return 1. / (1 + numpy.exp(-x))

def loss_function(y, t): # L2 Loss
    return numpy.mean((y - t) ** 2)

def network_output(x, w_1, w_2):
    h = sigmoid(w_1.dot(x))
    h[0] = 1. # bias
    y = w_2.dot(h)
    return y, h

def compute_gradient_layer_1(x, w, h, y, t):
    tmp = numpy.outer(w, y - t) * h * (1 - h)
    return tmp.dot(x.T) / len(x)

def compute_gradient_layer_2(h, y, t):
    return numpy.mean((y - t) * h, axis=1)

def gradient_descent(x, t, w_1, w_2, eta, epochs):
    loss_array = numpy.zeros(epochs)
    
    for epoch in range(epochs):
        y, h = network_output(x, w_1, w_2)
        w_1 -= eta * compute_gradient_layer_1(x, w_2, h, y, t)
        w_2 -= eta * compute_gradient_layer_2(h, y, t)
        y, h = network_output(x, w_1, w_2)
        loss_array[epoch] = loss_function(y, t)
    
    return w_1, w_2, loss_array

In [3]:
# prepare dataset
df = pandas.read_csv("data/student-mat.csv", sep=";")
drop_columns = ["Mjob", "Fjob", "reason", "guardian"]
df.drop(drop_columns, inplace=True, axis=1)

# convert to binary
df["school"].replace({"GP": -1, "MS": 1}, inplace=True)
df["sex"].replace({"F": -1, "M": 1}, inplace=True)
df["address"].replace({"U": -1, "R": 1}, inplace=True)
df["famsize"].replace({"GT3": -1, "LE3": 1}, inplace=True)
df["Pstatus"].replace({"A": -1, "T": 1}, inplace=True)
df["schoolsup"].replace({"no": -1, "yes": 1}, inplace=True)
df["famsup"].replace({"no": -1, "yes": 1}, inplace=True)
df["paid"].replace({"no": -1, "yes": 1}, inplace=True)
df["activities"].replace({"no": -1, "yes": 1}, inplace=True)
df["nursery"].replace({"no": -1, "yes": 1}, inplace=True)
df["higher"].replace({"no": -1, "yes": 1}, inplace=True)
df["internet"].replace({"no": -1, "yes": 1}, inplace=True)
df["romantic"].replace({"no": -1, "yes": 1}, inplace=True)

# convert all values to float
df = df.astype(float)

# init input & target
T = df[df.columns[-3:]].to_numpy()
X = df[df.columns[:-3]].to_numpy()





In [4]:
X

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,traveltime,studytime,...,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences
0,-1.0,-1.0,18.0,-1.0,-1.0,-1.0,4.0,4.0,2.0,2.0,...,1.0,-1.0,-1.0,4.0,3.0,4.0,1.0,1.0,3.0,6.0
1,-1.0,-1.0,17.0,-1.0,-1.0,1.0,1.0,1.0,1.0,2.0,...,1.0,1.0,-1.0,5.0,3.0,3.0,1.0,1.0,3.0,4.0
2,-1.0,-1.0,15.0,-1.0,1.0,1.0,1.0,1.0,1.0,2.0,...,1.0,1.0,-1.0,4.0,3.0,2.0,2.0,3.0,3.0,10.0
3,-1.0,-1.0,15.0,-1.0,-1.0,1.0,4.0,2.0,1.0,3.0,...,1.0,1.0,1.0,3.0,2.0,2.0,1.0,1.0,5.0,2.0
4,-1.0,-1.0,16.0,-1.0,-1.0,1.0,3.0,3.0,1.0,2.0,...,1.0,-1.0,-1.0,4.0,3.0,2.0,1.0,2.0,5.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390,1.0,1.0,20.0,-1.0,1.0,-1.0,2.0,2.0,1.0,2.0,...,1.0,-1.0,-1.0,5.0,5.0,4.0,4.0,5.0,4.0,11.0
391,1.0,1.0,17.0,-1.0,1.0,1.0,3.0,1.0,2.0,1.0,...,1.0,1.0,-1.0,2.0,4.0,5.0,3.0,4.0,2.0,3.0
392,1.0,1.0,21.0,1.0,-1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,-1.0,-1.0,5.0,5.0,3.0,3.0,3.0,3.0,3.0
393,1.0,1.0,18.0,1.0,1.0,1.0,3.0,2.0,3.0,1.0,...,1.0,1.0,-1.0,4.0,4.0,1.0,3.0,4.0,5.0,0.0


In [7]:
df

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,traveltime,studytime,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,-1.0,-1.0,18.0,-1.0,-1.0,-1.0,4.0,4.0,2.0,2.0,...,4.0,3.0,4.0,1.0,1.0,3.0,6.0,5.0,6.0,6.0
1,-1.0,-1.0,17.0,-1.0,-1.0,1.0,1.0,1.0,1.0,2.0,...,5.0,3.0,3.0,1.0,1.0,3.0,4.0,5.0,5.0,6.0
2,-1.0,-1.0,15.0,-1.0,1.0,1.0,1.0,1.0,1.0,2.0,...,4.0,3.0,2.0,2.0,3.0,3.0,10.0,7.0,8.0,10.0
3,-1.0,-1.0,15.0,-1.0,-1.0,1.0,4.0,2.0,1.0,3.0,...,3.0,2.0,2.0,1.0,1.0,5.0,2.0,15.0,14.0,15.0
4,-1.0,-1.0,16.0,-1.0,-1.0,1.0,3.0,3.0,1.0,2.0,...,4.0,3.0,2.0,1.0,2.0,5.0,4.0,6.0,10.0,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390,1.0,1.0,20.0,-1.0,1.0,-1.0,2.0,2.0,1.0,2.0,...,5.0,5.0,4.0,4.0,5.0,4.0,11.0,9.0,9.0,9.0
391,1.0,1.0,17.0,-1.0,1.0,1.0,3.0,1.0,2.0,1.0,...,2.0,4.0,5.0,3.0,4.0,2.0,3.0,14.0,16.0,16.0
392,1.0,1.0,21.0,1.0,-1.0,1.0,1.0,1.0,1.0,1.0,...,5.0,5.0,3.0,3.0,3.0,3.0,3.0,10.0,8.0,7.0
393,1.0,1.0,18.0,1.0,1.0,1.0,3.0,2.0,3.0,1.0,...,4.0,4.0,1.0,3.0,4.0,5.0,0.0,11.0,12.0,10.0


In [6]:
df.Pstatus.unique()

array([-1.,  1.])